In [19]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
from experiments.exp_basic import Exp_Basic
from experiments.exp_long_term_forecasting_partial import Exp_Long_Term_Forecast_Partial
from experiments.exp_long_term_forecasting import Exp_Long_Term_Forecast
from data_provider.data_factory import data_provider
from model.iTransformer import Model
import random
import torch
import sys
import os
import re

fix_seed = 2024
random.seed(fix_seed)
torch.manual_seed(fix_seed)
np.random.seed(fix_seed)

In [20]:
model = Model()
model.load_state_dict(torch.load('checkpoints/custom-model_iTransformer_custom_S_ft96_sl48_ll96_pl128_dm8_nh2_el1_dl128_df1_fctimeF_ebTrue_dtExp_projection_0/checkpoint.pth'))

TypeError: Model.__init__() missing 1 required positional argument: 'configs'

In [6]:
mae, mse, rmse, mape, mspe = np.load(os.path.join('results/' + setting, 'metrics.npy'))
preds = np.load(os.path.join('results/' + setting, 'pred.npy'))
trues = np.load(os.path.join('results/' + setting, 'true.npy'))
train_loss = np.load(os.path.join('results/' + setting, 'train_loss.npy'))
val_loss = np.load(os.path.join('results/' + setting, 'val_loss.npy'))

In [8]:
trues

array([[[0.5891405 ],
        [0.5891405 ],
        [0.5891405 ],
        ...,
        [0.47864905],
        [0.47864905],
        [0.5368024 ]],

       [[0.5891405 ],
        [0.5891405 ],
        [0.5891405 ],
        ...,
        [0.47864905],
        [0.5368024 ],
        [0.5019104 ]],

       [[0.5891405 ],
        [0.5891405 ],
        [0.5891405 ],
        ...,
        [0.5368024 ],
        [0.5019104 ],
        [0.4902797 ]],

       ...,

       [[2.5430944 ],
        [2.54891   ],
        [2.54891   ],
        ...,
        [2.4791257 ],
        [2.3860803 ],
        [2.4093418 ]],

       [[2.54891   ],
        [2.54891   ],
        [2.5721712 ],
        ...,
        [2.3860803 ],
        [2.4093418 ],
        [2.4093418 ]],

       [[2.54891   ],
        [2.5721712 ],
        [2.4558644 ],
        ...,
        [2.4093418 ],
        [2.4093418 ],
        [2.4093418 ]]], dtype=float32)

In [9]:
train_loss

array([0.68322264, 0.60529286, 0.57409283, 0.56181302, 0.55694538])

In [10]:
plt.plot(train_loss)

plt.title('Training Loss')
plt.show()

In [11]:
plt.plot(val_loss)

plt.title('Validation Loss')
plt.show()